# Day 12

## Part I

第一部分按照题目要求写代码即可，这里使用了两个枚举量，一个代表方向，一个代表转向：

In [1]:
from enum import Enum

Directions = Enum('Directions', ('east', 'south', 'west', 'north'), start = 0)

Turn = Enum('Turn', ('left', 'right'))

下面定义核心逻辑类Ferry，实现移动、前进和转向的逻辑，其中转向的逻辑使用了整数向枚举的转换：

In [2]:
class Ferry(object):
    def __init__(self):
        self.facing = Directions.east
        self.x = 0
        self.y = 0
    def move_direction(self, d: Directions, value: int):
        if d == Directions.east:
            self.x += value
        elif d == Directions.west:
            self.x -= value
        elif d == Directions.north:
            self.y += value
        elif d == Directions.south:
            self.y -= value
    def forward(self, value: int):
        self.move_direction(self.facing, value)
    def turn(self, d: Turn, value: int):
        # 转向的角度整除90度，然后设置成船的新朝向
        new_d = self.facing.value + value // 90 if d == Turn.right else self.facing.value - value // 90
        self.facing = Directions(new_d % 4)
    def manhatten_distance(self):
        return abs(self.x) + abs(self.y)
    def do_instruction(self, inst: str, value: int):
        # 这个方法用来执行每一行的指令
        if inst == 'N':
            self.move_direction(Directions.north, value)
        elif inst == 'S':
            self.move_direction(Directions.south, value)
        elif inst == 'E':
            self.move_direction(Directions.east, value)
        elif inst == 'W':
            self.move_direction(Directions.west, value)
        elif inst == 'F':
            self.forward(value)
        elif inst == 'L':
            self.turn(Turn.left, value)
        elif inst == 'R':
            self.turn(Turn.right, value)

读取和处理输入数据，放置到一个字符串和整数组成的元组列表中：

In [3]:
from typing import List, Tuple

def read_input(input_file: str) -> List[Tuple[str, int]]:
    with open(input_file) as fn:
        return [(line[0], int(line.rstrip()[1:])) for line in fn.readlines()]

第一部分的问题，定义为函数：

In [4]:
def part1_solution(input_file: str) -> int:
    ferry = Ferry()
    for inst, value in read_input(input_file):
        ferry.do_instruction(inst, value)
    return ferry.manhatten_distance()

单元测试：

In [5]:
assert(part1_solution('testcase1.txt') == 25)

第一部分的结果：

In [6]:
part1_solution('input.txt')

1533

## Part II

第二部分修改了大部分的逻辑，因此下面重新定义了一个Ferry2类来处理核心逻辑，由于不再关心船只的朝向，转而关注waypoint的位置，因此代码中直接使用了复数来表示船只位置和waypoint的相对位置。用复数的好处是前进逻辑只需要用复数运算即可完成，并且转向逻辑更加简单，右转90度就是乘上-1j，左转90度就是乘上1j：

In [7]:
class Ferry2(object):
    def __init__(self):
        self.position = 0 + 0j
        self.waypoint = 10 + 1j
    def move_waypoint(self, d: Directions, value: int):
        if d == Directions.east:
            self.waypoint += value
        elif d == Directions.west:
            self.waypoint -= value
        elif d == Directions.north:
            self.waypoint += value * 1j
        elif d == Directions.south:
            self.waypoint -= value * 1j
    def forward(self, value: int):
        self.position += value * self.waypoint
    def turn(self, d: Turn, value: int):
        for _ in range(value // 90 % 4):
            if d == Turn.left:
                self.waypoint *= 1j
            else:
                self.waypoint *= -1j
    def manhatten_distance(self):
        return abs(int(self.position.real)) + abs(int(self.position.imag))
    def do_instruction(self, inst: str, value: int):
        if inst == 'N':
            self.move_waypoint(Directions.north, value)
        elif inst == 'S':
            self.move_waypoint(Directions.south, value)
        elif inst == 'E':
            self.move_waypoint(Directions.east, value)
        elif inst == 'W':
            self.move_waypoint(Directions.west, value)
        elif inst == 'F':
            self.forward(value)
        elif inst == 'L':
            self.turn(Turn.left, value)
        elif inst == 'R':
            self.turn(Turn.right, value)

第一部分求结果的函数：

In [8]:
def part2_solution(input_file: str) -> int:
    ferry = Ferry2()
    for inst, value in read_input(input_file):
        ferry.do_instruction(inst, value)
    return ferry.manhatten_distance()

单元测试

In [9]:
assert(part2_solution('testcase1.txt') == 286)

第二部分的结果：

In [10]:
part2_solution('input.txt')

25235